***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [ ]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME               PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-206259848  GCE       4                                       RUNNING  us-central1-a


# Imports & Setup

In [ ]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [ ]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Dec  7 13:17 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [ ]:
spark

In [ ]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'search_engine_bucket' 
client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    print(b.name)

graphframes.sh


***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [ ]:
full_path = "gs://wikidata_preprocessed/*"
parquetFile = spark.read.parquet(full_path)
doc_title_pairs = parquetFile.select("id", "title").rdd

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [ ]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_title_gcp.py

inverted_index_gcp.py


In [ ]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_title_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [ ]:
from inverted_index_title_gcp import InvertedIndex

**YOUR TASK (10 POINTS)**: Use your implementation of `word_count`, `reduce_word_counts`, `calculate_df`, and `partition_postings_and_write` functions from Colab to build an inverted index for all of English Wikipedia in under 2 hours.

A few notes: 
1. The number of corpus stopwords below is a bit bigger than the colab version since we are working on the whole corpus and not just on one file.
2. You need to slightly modify your implementation of  `partition_postings_and_write` because the signature of `InvertedIndex.write_a_posting_list` has changed and now includes an additional argument called `bucket_name` for the target bucket. See the module for more details.
3. You are not allowed to change any of the code not coming from Colab. 

In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

# PLACE YOUR CODE HERE

##################################################################################################

def word_count(text, id):
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  filtered_tokens = [tok for tok in tokens if tok not in all_stopwords]
  count = Counter(filtered_tokens)
  return [(k,(id,v)) for k,v in count.items()]

##################################################################################################

def reduce_word_counts(unsorted_pl):
  return sorted(unsorted_pl)

##################################################################################################

def calculate_df(postings):
  return postings.mapValues(lambda x: len(x))

##################################################################################################

def doc_title_mapping_creator(id, title):
  return [(id, title)]

##################################################################################################

def partition_postings_and_write(postings):
  posting_with_bucket = postings.map(lambda x: (token2bucket_id(x[0]), (x[0],x[1])))
  b_w_pl = posting_with_bucket.groupByKey().map(lambda x : (x[0], list(x[1])))
  posting_locs_dict_rdd = b_w_pl.map(lambda x: InvertedIndex.write_a_posting_list(x, bucket_name))
  return posting_locs_dict_rdd

##################################################################################################

In [ ]:
# time the index creation time
t_start = time()

# word counts map
word_count_title = doc_title_pairs.flatMap(lambda x: word_count(x[1], x[0]))

# create postings and calculate df
postings_title = word_count_title.groupByKey().mapValues(reduce_word_counts)
df_dictionary = calculate_df(postings_title).collectAsMap()

#create docID - title dictionary
doc_title_dictionary = doc_title_pairs.flatMap(lambda x: doc_title_mapping_creator(x.id, x.title)).collectAsMap()

# partition posting lists and write out
_ = partition_postings_and_write(postings_title).collect()

index_const_time = time() - t_start

In [ ]:
# test index construction time
index_const_time

In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='title_index'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

Putting it all together

In [ ]:
# Create inverted index instance
inverted = InvertedIndex()

# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs

# Add the token - df dictionary to the inverted index
inverted.df = df_dictionary

# Add the doc_id - title dictionary to the inverted index
inverted.doc_title_mapping = doc_title_dictionary

# write the global stats out
inverted.write_index('.', 'title_index')

# upload to gs
index_src = "title_index.pkl"
index_dst = f'gs://{bucket_name}/title_index/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://index.pkl [Content-Type=application/octet-stream]...
/ [1 files][ 18.4 MiB/ 18.4 MiB]                                                
Operation completed over 1 objects/18.4 MiB.                                     


In [ ]:
!gsutil ls -lh $index_dst

 18.45 MiB  2021-12-07T15:02:53Z  gs://bucket_206259848/postings_gcp/index.pkl
TOTAL: 1 objects, 19343455 bytes (18.45 MiB)


# Reporting

**YOUR TASK (5 points):** execute and complete the following lines to complete 
the reporting requirements for assignment #3. 

In [ ]:
# size of input data
!gsutil du -sh "gs://wikidata_preprocessed/"

14.28 GiB    gs://wikidata_preprocessed


In [ ]:
# size of index data
index_dst = f'gs://{bucket_name}/title_index/'
!gsutil du -sh "$index_dst"

5.92 GiB     gs://bucket_206259848/postings_gcp
